In [24]:
import numpy as np
import pandas as pd
import datetime
from pandas_profiling import ProfileReport
from datetime import datetime ,timedelta,date
pd.options.mode.chained_assignment = None
import lifetimes
pd.set_option('display.float_format', '{:.2E}'.format)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [25]:
from lifetimes.utils import *
from lifetimes import BetaGeoFitter
from lifetimes import ParetoNBDFitter
from lifetimes.plotting import plot_probability_alive_matrix, plot_frequency_recency_matrix
from lifetimes.generate_data import beta_geometric_nbd_model
import matplotlib.pyplot as plt
from lifetimes.plotting import plot_calibration_purchases_vs_holdout_purchases, plot_period_transactions,plot_history_alive

bgf = BetaGeoFitter()

In [26]:
def get_run_date():
    base = datetime.today()
    current_date = base.strftime("%Y-%m-%d")
    return current_date

In [27]:
df=pd.read_csv('_SELECT_a_b_FROM_ENAIRA_KYCPERSONAL_a_INNER_JOIN_ENAIRA_CONSOLID_202306061840.csv',low_memory=False)

In [28]:
df.head(3)

,ID,uid,UID_TYPE,WALLET_TYPE,RESULT,USER_ACCOUNT,ASSOCIATION,PHONE,EMAIL_ID,TAGS,QUEUE,STAGE,RULE_ID,POLICY_RESULT,PRIORITY,PROGRESS,ASSIGNEE,FIRST_NAME,LAST_NAME,MIDDLE_NAME,TITLE,OWNERSHIP,PROFILE,LAST_STAGE,LAST_REPORTED_RULE,ACCOUNT_NUMBER_AND_INVITE_CODE,POSTAL_CODE,TOWN,STATE,PARISH,ADDRESS,COUNTRY_CODE,CUSTOMER_RISK_RATING,TIER,INVITE_CODE,BVN,ACCOUNT_NUMBER,TAX_IDENTIFICATION_NUMBER,COUNTRY_OF_BIRTH,STATE_OF_ORIGIN,PEP,IN_BLACKBOOK,SANCTION_SCREEN,WATCHLISTED,DESCRIPTION,CREATED_AT,CREATED_BY,LAST_UPDATED_AT,LAST_UPDATED_BY,SUBMISSION_TYPE,INST_CODE,NUBAN,DATE_OF_BIRTH,AML_STATUS,WALLET_ALIAS,WALLET_ADDRESS,WALLET_INFO,REQUEST_SRC,PASSWORD,REFERRERS_CODE,NIN,BUSINESS_NAME,PHONE_NUMBER,TEMP_PASSWORD,NEEDS_PASSWORD_RESET,AMOUNT,DESTINATION_ADDRESS,DESTINATION_WALLET_TYPE,RECEIPT_ID,SOURCE_ADDRESS,SOURCE_WALLET_TYPE,TRANSACTION_DATE,TRANSACTION_ID,TRANSACTION_STATUS,TRANSACTION_TYPE
0,852014,22530391713,BVN,consumer,ACCEPTED,kanolagos003@gmail.com,NaN,09180450548,kanolagos003@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HASHIM,ALHASSAN,NaN,Mr,NaN,NaN,NaN,NaN,NaN,NaN,195,26.000,JI019,gagarawa,NG,0.000,2,NaN,22530391713.000,2209742079,NaN,NG,26,NaN,NaN,NaN,False,NaN,2022-09-28 08:27:03.271,enairamobilewallet,2022-09-28 08:27:03.271,enairamobilewallet,APP,57,1003293160,1997-05-12 00:00:00.000,AMLS02,@halhass.01ge1,01GE1K9TKPEP4NZSVS00PP92RJ,"{""tier"": ""2"", ""nuban"": ""1003293160"", ""message""...",NaN,be3747234cf4fe011cd9a2c93def901d,@gsalisu.01gdxy,NaN,NaN,NaN,NaN,NaN,20000,01G9CX7FCW2NXYSVE6GM9AHJPS,COMMERCE_WALLET,01GE2CQRFF6MR30F8BHM2G0Y2C,01GE1K9TKPEP4NZSVS00PP92RJ,COMMERCE_WALLET,2022-09-28T15:51:37.531308,01GE2CQR9V3FKTJ5Z9G4AB7KDG,complete,P2P
1,835860,22290304101,BVN,consumer,ACCEPTED,agrenska@nbobd.com,NaN,04937507678,agrenska@nbobd.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IDRISU,GANA,NaN,Mr,NaN,NaN,NaN,NaN,NaN,NaN,115,16.000,NG007,MOKWA,NG,0.000,2,NaN,22290304101.000,2059883292,NaN,NG,10,NaN,NaN,NaN,False,NaN,2022-09-27 12:32:56.140,enairamobilewallet,2022-09-27 12:32:56.140,enairamobilewallet,APP,33,1003130724,1955-10-01 00:00:00.000,AMLS02,@igana.01gdze,01GDZEZCK8KSSCN10JCJKQ70WM,"{""tier"": ""2"", ""nuban"": ""1003130724"", ""message""...",NaN,19181b607c0e6930ddee33bf5e6df2b5,NaN,NaN,NaN,NaN,NaN,NaN,20000,01G9CX7FCW2NXYSVE6GM9AHJPS,COMMERCE_WALLET,01GDZFJ9Q1SE1P1R910FTY1ZZB,01GDZEZCK8KSSCN10JCJKQ70WM,COMMERCE_WALLET,2022-09-27T12:36:07.288930,01GDZF51ZRYW7TAVXNQWV8AKD4,complete,P2P
2,837226,22260565079,BVN,consumer,ACCEPTED,beethe710@hamham.uk,NaN,08060541500,beethe710@hamham.uk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Haruna,Usman,NaN,Mr,NaN,NaN,NaN,NaN,NaN,NaN,682,16.000,NG003,emi wambai massalachi bologi area bida,NG,0.000,2,NaN,22260565079.000,3059131237,NaN,NG,16,NaN,NaN,NaN,False,NaN,2022-09-27 13:36:26.429,enairamobilewallet,2022-09-27 13:36:26.429,enairamobilewallet,APP,11,1003144406,1985-12-31 00:00:00.000,AMLS02,@husman.01gdzj,01GDZJKMNTBZB7B7099GX9YEG2,"{""tier"": ""2"", ""nuban"": ""1003144406"", ""message""...",NaN,b7ab7f991b8accc0accef92894de2236,NaN,NaN,NaN,NaN,NaN,NaN,20000,01G9CX7FCW2NXYSVE6GM9AHJPS,COMMERCE_WALLET,01GDZKH98QWSJV1WS3J47H066K,01GDZJKMNTBZB7B7099GX9YEG2,COMMERCE_WALLET,2022-09-27T13:52:33.499743,01GDZKH0PVN2CDD1CKP4KT7SK5,complete,P2P


In [29]:
data=df[['ID','uid','TITLE','CREATED_AT','STATE','AMOUNT', 'DESTINATION_ADDRESS','TIER'
      , 'SOURCE_ADDRESS', 'TRANSACTION_DATE',
'DATE_OF_BIRTH','LAST_UPDATED_AT']].copy()

In [30]:
data.head(5)

,ID,uid,TITLE,CREATED_AT,STATE,AMOUNT,DESTINATION_ADDRESS,TIER,SOURCE_ADDRESS,TRANSACTION_DATE,DATE_OF_BIRTH,LAST_UPDATED_AT
0,852014,22530391713,Mr,2022-09-28 08:27:03.271,26.000,20000,01G9CX7FCW2NXYSVE6GM9AHJPS,2,01GE1K9TKPEP4NZSVS00PP92RJ,2022-09-28T15:51:37.531308,1997-05-12 00:00:00.000,2022-09-28 08:27:03.271
1,835860,22290304101,Mr,2022-09-27 12:32:56.140,16.000,20000,01G9CX7FCW2NXYSVE6GM9AHJPS,2,01GDZEZCK8KSSCN10JCJKQ70WM,2022-09-27T12:36:07.288930,1955-10-01 00:00:00.000,2022-09-27 12:32:56.140
2,837226,22260565079,Mr,2022-09-27 13:36:26.429,16.000,20000,01G9CX7FCW2NXYSVE6GM9AHJPS,2,01GDZJKMNTBZB7B7099GX9YEG2,2022-09-27T13:52:33.499743,1985-12-31 00:00:00.000,2022-09-27 13:36:26.429
3,838186,22469277432,Mr,2022-09-27 14:17:30.785,13.000,20000,01G9CX7FCW2NXYSVE6GM9AHJPS,2,01GDZMYW2Y7EN773KP16V128ST,2022-09-27T14:28:02.120461,1975-04-07 00:00:00.000,2022-09-27 14:17:30.785
4,838910,22581014021,Mr,2022-09-27 14:50:08.823,3.000,20000,01G9CX7FCW2NXYSVE6GM9AHJPS,2,01GDZPTJ4027Q62F4RC1JZ95GS,2022-09-27T14:52:15.943603,1999-05-12 00:00:00.000,2022-09-27 14:50:08.823


In [31]:
def datatype_conv(data):   
    data['ID']=data['ID'].astype(str)
    data['uid']=data['uid'].astype(str)
    data['TIER']=data['TIER'].astype(str)
    data['STATE']=data['STATE'].astype(str)
    data['TRANSACTION_DATE']=pd.to_datetime(data['TRANSACTION_DATE'])
    data['CREATED_AT']=pd.to_datetime(data['CREATED_AT'])
    data['LAST_UPDATED_AT']=pd.to_datetime(data['LAST_UPDATED_AT'])
    data['TRANS_DATE_DATE']=data['TRANSACTION_DATE'].dt.date
    data['LAST_UPDATED_AT_DATE']=data['LAST_UPDATED_AT'].dt.date
    data['TRANSACTION_DATE']=pd.to_datetime(data['TRANSACTION_DATE'])
    data['DATE_OF_BIRTH']=pd.to_datetime(data['DATE_OF_BIRTH'])
    return data

In [32]:
data= datatype_conv(data)

In [33]:
data.head(5)

,ID,uid,TITLE,CREATED_AT,STATE,AMOUNT,DESTINATION_ADDRESS,TIER,SOURCE_ADDRESS,TRANSACTION_DATE,DATE_OF_BIRTH,LAST_UPDATED_AT,TRANS_DATE_DATE,LAST_UPDATED_AT_DATE
0,852014,22530391713,Mr,2022-09-28 08:27:03.271,26.0,20000,01G9CX7FCW2NXYSVE6GM9AHJPS,2,01GE1K9TKPEP4NZSVS00PP92RJ,2022-09-28 15:51:37.531308,1997-05-12,2022-09-28 08:27:03.271,2022-09-28,2022-09-28
1,835860,22290304101,Mr,2022-09-27 12:32:56.140,16.0,20000,01G9CX7FCW2NXYSVE6GM9AHJPS,2,01GDZEZCK8KSSCN10JCJKQ70WM,2022-09-27 12:36:07.288930,1955-10-01,2022-09-27 12:32:56.140,2022-09-27,2022-09-27
2,837226,22260565079,Mr,2022-09-27 13:36:26.429,16.0,20000,01G9CX7FCW2NXYSVE6GM9AHJPS,2,01GDZJKMNTBZB7B7099GX9YEG2,2022-09-27 13:52:33.499743,1985-12-31,2022-09-27 13:36:26.429,2022-09-27,2022-09-27
3,838186,22469277432,Mr,2022-09-27 14:17:30.785,13.0,20000,01G9CX7FCW2NXYSVE6GM9AHJPS,2,01GDZMYW2Y7EN773KP16V128ST,2022-09-27 14:28:02.120461,1975-04-07,2022-09-27 14:17:30.785,2022-09-27,2022-09-27
4,838910,22581014021,Mr,2022-09-27 14:50:08.823,3.0,20000,01G9CX7FCW2NXYSVE6GM9AHJPS,2,01GDZPTJ4027Q62F4RC1JZ95GS,2022-09-27 14:52:15.943603,1999-05-12,2022-09-27 14:50:08.823,2022-09-27,2022-09-27


In [44]:
import pandas as pd

def calculate_rfm(data, date_column):
    """
    frequency - the number of repeat purchases (more than 1 purchases)
    recency - the time between the first and the last transaction
    T - the time between the first purchase and the end of the transaction period
    monetary_value - it is the mean of a given customer's sales value

    The Recency calculation is modified to find the difference
    in days between the maximum date (NOW) and the transaction date (x),
    representing the number of days since the last transaction.

    Calculating T - Determine the first purchase date for each SOURCE_ADDRESS.
    Calculate the difference in days between the first purchase date and the end of the transaction period.
    Add the calculated T values as a new column in the rfm_table dataframe.
    """

    max_date = data[date_column].max()

    # Calculate Recency, Frequency, Monetary, and T
    rfm_table = data.groupby('SOURCE_ADDRESS').agg(
         Recency=('TRANS_DATE_DATE', lambda x: (max_date - x.max()).days),
        Frequency=('TRANS_DATE_DATE', 'count'),
        Monetary=('AMOUNT', 'mean'),
        FirstPurchase=('TRANS_DATE_DATE', 'min')
    ).reset_index()

    rfm_table['T'] = (max_date - rfm_table['FirstPurchase']).dt.days
    rfm_table.loc[rfm_table['Recency'] == 0, 'Recency'] = 1
    

    return rfm_table.drop(columns=['FirstPurchase'])


In [45]:
rfm_table=calculate_rfm(data, 'TRANS_DATE_DATE')

In [46]:
rfm_table3=rfm_table.copy()

In [47]:
bgf = lifetimes.BetaGeoFitter(penalizer_coef=0.0)
bgf.fit(rfm_table3['Frequency'], rfm_table3['Recency'], rfm_table3['T'])

  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 0.33187349835320046
        x: [ 9.448e-01  3.116e-01  1.022e+01  1.423e+01]
      nit: 44
      jac: [-1.124e-05  6.907e-06  1.195e-07 -1.402e-07]
 hess_inv: [[ 6.261e+00  7.142e+00  3.579e+00  1.947e-01]
            [ 7.142e+00  8.988e+00  3.773e+00  2.053e-01]
            [ 3.579e+00  3.773e+00  2.501e+02  1.362e+01]
            [ 1.947e-01  2.053e-01  1.362e+01  7.504e-01]]
     nfev: 115
     njev: 104


ConvergenceError: 
The model did not converge. Try adding a larger penalizer to see if that helps convergence.
